In [5]:
import pandas as pd
import numpy as np
from langchain_core.messages import HumanMessage,AIMessage,SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

### 一、测试LangChain内置代码解释器工具功能

In [6]:
dat = pd.read_csv("data/WA_Fn-UseC_-Telco-Customer-Churn.csv",header=0)

from langchain_experimental.tools import PythonAstREPLTool
# langchain内置的python解释器，将其功能传递给python_tool变量。
python_tool = PythonAstREPLTool(locals={"df":dat})
python_tool.invoke("df['SeniorCitizen'].mean()")

model = ChatOllama(
    model = 'llama3.1:8b',
    # model = 'deepseek-r1:1.5b', 没有调用tool功能
    base_url = 'http://localhost:11434/'
)

In [7]:
model

ChatOllama(model='llama3.1:8b', base_url='http://localhost:11434/')

In [8]:
# bind_tools 是一个用于将工具（Tools）绑定到可调用对象（如 LLM 或 Chain）上的方法，使得模型能够更好地与外部工具交互。
model_with_tool = model.bind_tools([python_tool])
response = model_with_tool.invoke("请分析'df'这张表，计算有数值的列的平均值。")

In [6]:
# 查看response输出，可以看到content内容是空的
# ✅ 模型 确实识别并调用了绑定的工具（python_repl_ast）
# ✅ df.describe() 是模型产生的代码
# ❌ 模型没有返回“人类可读的回答”，即 content 是空的
# 调用的是 model_with_tool.invoke(...)，只触发 tool_call，但未执行该工具。
response

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-07-14T07:22:46.0370326Z', 'done': True, 'done_reason': 'stop', 'total_duration': 32125815500, 'load_duration': 9169297400, 'prompt_eval_count': 216, 'prompt_eval_duration': 11817326300, 'eval_count': 46, 'eval_duration': 11098755200, 'model_name': 'llama3.1:8b'}, id='run--50ebee79-334e-43b8-aa60-411494600670-0', tool_calls=[{'name': 'python_repl_ast', 'args': {'query': "import pandas as pd\npd.DataFrame({'A': [1, 2], 'B': [3, 4]}).describe()"}, 'id': '22e5fa95-37f5-48ae-98e3-ecf4439eeffa', 'type': 'tool_call'}], usage_metadata={'input_tokens': 216, 'output_tokens': 46, 'total_tokens': 262})

In [ ]:
from langchain_core.output_parsers.openai_tools import JsonOutputKeyToolsParser
parser = JsonOutputKeyToolsParser(key_name=python_tool.name, first_tool_only=True)
model_chain = model_with_tool | parser
response = model_chain.invoke("请分析'df'这张表，计算有数值的列的平均值。")
response

In [8]:
# JsonOutputKeyToolsParser 从大模型的 function calling / tool calling 结构中，提取你关心的字段（key），并转换为 Python dict，供下一步使用（比如代码执行）。
print(parser)
print(model_chain)

first_tool_only=True key_name='python_repl_ast'
first=RunnableBinding(bound=ChatOllama(model='llama3.1:8b', base_url='http://localhost:11434/'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'python_repl_ast', 'description': 'A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.', 'parameters': {'properties': {'query': {'description': 'code snippet to run', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}]}, config={}, config_factories=[]) middle=[] last=JsonOutputKeyToolsParser(first_tool_only=True, key_name='python_repl_ast')


In [10]:
# 所以接下来给出提示词模板，告知大模型需要操作
system = f"""
你可以访问一个名为 `df` 的 pandas 数据框，你可以使用df.head().to_markdown() 查看数据集的基本信息， \
请根据用户提出的问题，编写 Python 代码来回答。只返回代码，不返回其他内容。只允许使用 pandas 和内置库。
"""
prompt = ChatPromptTemplate([
    ("system", system),
    ("user", "{question}")
])
code_chain = prompt | model_with_tool | parser

通过 prompt <font color=red> 向模型提问 → 模型识别你希望它生成可执行代码 → 模型以 Tool 调用的形式返回 {"query": "..."} → JsonOutputKeyToolsParser 负责提取出这段代码</font></br>
最终输出了：{'query': "df.select_dtypes(include=['number']).mean()"}

In [10]:
code_chain.invoke({"question": "请分析'df'这张表，计算有数值的列的平均值。"})

{'query': "df.select_dtypes(include=['number']).mean()"}

In [13]:
chain = code_chain | python_tool
chain.invoke({"question": "请分析'df'这张表，计算有数值的列的平均值。"})

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [14]:
chain.invoke({"question": "分析'df'这张表，gender统计信息，每个类别的总数"})

'gender\nMale      3555\nFemale    3488\nName: count, dtype: int64\n'

In [11]:
# 将函数封装成Langchain可以识别的对象
from langchain_core.runnables import RunnableLambda
def code_print(res):
    print("即将运行的python代码是", res['query'])
    return res
print_code = RunnableLambda(code_print)
print_code_chain = prompt | model_with_tool | parser | print_code | python_tool

In [13]:
print_code_chain.invoke({'question': '请计算包含数字的列的平均值'})

即将运行的python代码是 df.select_dtypes(include=['number']).mean()


SeniorCitizen      0.162147
tenure            32.371149
MonthlyCharges    64.761692
dtype: float64

### 二、Langchain 接入自定义的外部工具

In [16]:
# 导入openweather的api key
import os
from dotenv import load_dotenv
load_dotenv(override=True)

openweather_api = os.getenv("OPENWEATHER_API_KEY")

In [19]:
import json,requests
def get_weather(loc):
    """
    查询实时天气函数
    ：param loc：城市名称；
    返回结果是json对象，包含了全部重要的天气信息。
    """
    url = "https://api.openweathermap.org/data/2.5/weather"

    # 构建参数
    params = {
        'q': loc,
        'appid': openweather_api,
        'units': 'metric', # 使用摄氏度
        'lang': 'zh_cn' # 输出问简体中文
    }

    # 发送请求
    response = requests.get(url, params=params)

    # 解析响应
    # 从 HTTP 响应中解析 JSON 内容，通常是一个 dict 或 list 对象。
    data = response.json()
    # 把 Python 对象（如 dict 或 list）转换成 JSON 格式的字符串
    return json.dumps(data)

In [20]:
get_weather('Wuhan')

'{"coord": {"lon": 114.2667, "lat": 30.5833}, "weather": [{"id": 804, "main": "Clouds", "description": "\\u9634\\uff0c\\u591a\\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 34.12, "feels_like": 39.36, "temp_min": 34.12, "temp_max": 34.12, "pressure": 1000, "humidity": 52, "sea_level": 1000, "grnd_level": 997}, "visibility": 10000, "wind": {"speed": 2.99, "deg": 252, "gust": 2.86}, "clouds": {"all": 97}, "dt": 1752481116, "sys": {"country": "CN", "sunrise": 1752442192, "sunset": 1752492435}, "timezone": 28800, "id": 1791247, "name": "Wuhan", "cod": 200}'

In [99]:
# 将比较复杂的外部函数添加到chain中，需要调用langchain里面的tool装饰器声明。
# 需要注意的是tool在声明的时候需要description，即函数解释，可以写在函数内部注释，也可以在@tool(description参数中描述)
from langchain_core.tools import tool
@tool
def get_weather(loc:str) -> str:
    """
    查询实时天气函数
    ：param loc：城市名称（使用城市的对应英文表示，例如输入的是“武汉”，loc对应的是Wuhan，“纽约”，loc对应的是"New York"）；
    返回结果是json对象，包含了全部重要的天气信息。
    """
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': loc,
        'appid': openweather_api,
        'units': 'metric', # 使用摄氏度
        'lang': 'zh_cn' # 输出问简体中文
    }
    response = requests.get(url, params=params)
    data = response.json()
    return json.dumps(data)

In [93]:
print(get_weather.name)
print(get_weather.args)
print(get_weather.description)
print(get_weather)

get_weather
{'loc': {'title': 'Loc', 'type': 'string'}}
查询实时天气函数
：param loc：城市名称（使用城市的对应英文表示，例如输入的是“武汉”，loc对应的是Wuhan，“纽约”，loc对应的是NewYork）；
返回结果是json对象，包含了全部重要的天气信息。
name='get_weather' description='查询实时天气函数\n：param loc：城市名称（使用城市的对应英文表示，例如输入的是“武汉”，loc对应的是Wuhan，“纽约”，loc对应的是NewYork）；\n返回结果是json对象，包含了全部重要的天气信息。' args_schema=<class 'langchain_core.utils.pydantic.get_weather'> func=<function get_weather at 0x000002A9C3E61E40>


In [100]:
# 定义天气查询工具，现在get_weather本身已经是一个工具对象了，直接赋值。
weather_tool = [get_weather]

# 将工具绑定给模型
weather_model_1 = model.bind_tools(weather_tool)

In [31]:
weather_model_1.invoke("请问武汉的天气怎么样？")

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-07-14T08:31:21.7545801Z', 'done': True, 'done_reason': 'stop', 'total_duration': 24632400000, 'load_duration': 9207636700, 'prompt_eval_count': 190, 'prompt_eval_duration': 10415706300, 'eval_count': 18, 'eval_duration': 5004656400, 'model_name': 'llama3.1:8b'}, id='run--081c87c5-42ab-48f8-a690-4595707e82b7-0', tool_calls=[{'name': 'get_weather', 'args': {'loc': '武汉'}, 'id': '63365a9c-27f3-48bc-ae90-143cd7123dcf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 190, 'output_tokens': 18, 'total_tokens': 208})

In [101]:
weather_model_2 = weather_model_1 | parser

In [57]:
weather_model_2.invoke("请问武汉的天气如何？")

{'loc': 'Wuhan'}

In [102]:
weather_model_3 = weather_model_2 | get_weather

In [64]:
weather_model_3

RunnableBinding(bound=ChatOllama(model='llama3.1:8b', base_url='http://localhost:11434/'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_weather', 'description': '查询实时天气函数\n：param loc：城市名称（使用城市的对应拼音表示，例如输入的是“武汉”，loc对应的是Wuhan）；\n返回结果是json对象，包含了全部重要的天气信息。', 'parameters': {'properties': {'loc': {'type': 'string'}}, 'required': ['loc'], 'type': 'object'}}}]}, config={}, config_factories=[])
| JsonOutputKeyToolsParser(first_tool_only=True, key_name='get_weather')
| StructuredTool(name='get_weather', description='查询实时天气函数\n：param loc：城市名称（使用城市的对应拼音表示，例如输入的是“武汉”，loc对应的是Wuhan）；\n返回结果是json对象，包含了全部重要的天气信息。', args_schema=<class 'langchain_core.utils.pydantic.get_weather'>, func=<function get_weather at 0x000002A9C0463A60>)

In [68]:
# 输出的是get_weather函数返回的json数据
weather_model_3.invoke("请问武汉的天气怎么样？")

'{"coord": {"lon": 114.2667, "lat": 30.5833}, "weather": [{"id": 804, "main": "Clouds", "description": "\\u9634\\uff0c\\u591a\\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 34.96, "feels_like": 41.96, "temp_min": 34.96, "temp_max": 34.96, "pressure": 999, "humidity": 59, "sea_level": 999, "grnd_level": 996}, "visibility": 10000, "wind": {"speed": 1.79, "deg": 233, "gust": 1.54}, "clouds": {"all": 98}, "dt": 1752482470, "sys": {"country": "CN", "sunrise": 1752442192, "sunset": 1752492435}, "timezone": 28800, "id": 1791247, "name": "Wuhan", "cod": 200}'

In [103]:
# 第二次传递模型，解析结果，输出人可读的结果。
# PromptTemplate和ChatPromptTemplate的区别：
    # PromptTemplate：构造一段完整的 prompt 文本，适合传统 text completion 模型
    # ChatPromptTemplate：构造一个消息式的聊天结构，适合 chat completion 模型
from langchain.prompts import PromptTemplate,ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
prompt = PromptTemplate.from_template(
"""
你将收到一段 JSON 格式的天气数据，请用简洁而自然的方式将其转述给用户。

以下是天气 JSON 数据：
```json
{weather_json}
```
请将其转化为中文天气描述，例如：
"北京当前天气为晴天，气温是23℃，湿度58%，风速2.1米/秒"
只返回描述，不需要解释和其他说明。
"""
)

In [104]:
# 构建输出结果解析链
output_chain = prompt | model | StrOutputParser()
output_chain

PromptTemplate(input_variables=['weather_json'], input_types={}, partial_variables={}, template='\n你将收到一段 JSON 格式的天气数据，请用简洁而自然的方式将其转述给用户。\n\n以下是天气 JSON 数据：\n```json\n{weather_json}\n```\n请将其转化为中文天气描述，例如：\n"北京当前天气为晴天，气温是23℃，湿度58%，风速2.1米/秒"\n只返回描述，不需要解释和其他说明。\n')
| ChatOllama(model='llama3.1:8b', base_url='http://localhost:11434/')
| StrOutputParser()

In [111]:
weather_final = weather_model_3 | output_chain
weather_final

RunnableBinding(bound=ChatOllama(model='llama3.1:8b', base_url='http://localhost:11434/'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_weather', 'description': '查询实时天气函数\n：param loc：城市名称（使用城市的对应英文表示，例如输入的是“武汉”，loc对应的是Wuhan，“纽约”，loc对应的是"New York"）；\n返回结果是json对象，包含了全部重要的天气信息。', 'parameters': {'properties': {'loc': {'type': 'string'}}, 'required': ['loc'], 'type': 'object'}}}]}, config={}, config_factories=[])
| JsonOutputKeyToolsParser(first_tool_only=True, key_name='get_weather')
| StructuredTool(name='get_weather', description='查询实时天气函数\n：param loc：城市名称（使用城市的对应英文表示，例如输入的是“武汉”，loc对应的是Wuhan，“纽约”，loc对应的是"New York"）；\n返回结果是json对象，包含了全部重要的天气信息。', args_schema=<class 'langchain_core.utils.pydantic.get_weather'>, func=<function get_weather at 0x000002A9C1E25940>)
| PromptTemplate(input_variables=['weather_json'], input_types={}, partial_variables={}, template='\n你将收到一段 JSON 格式的天气数据，请用简洁而自然的方式将其转述给用户。\n\n以下是天气 JSON 数据：\n```json\n{weather_json}\n```\n请将其转化为中文天气描述，例如：\n"北京当前天

In [91]:
weather_final.invoke("请问武汉天气如何？")

'"武汉当前天气为多云，气温34.96℃，湿度59%，风速1.79米/秒"'

In [106]:
# 返回的效果不是很好，需要检查是否中间层传递错误，需要更改限定词或者描述。
weather_final.invoke("请问纽约天气如何？")

'天气信息如下：\n\n新욕市当前天气为多云，气温23.45℃，感到的温度24.25℃，最低气温23.45℃，最高气温23.45℃。相对湿度达92%，大气压力为1017，海平面气压1017，地面气压1016。视觉距离10000米。风速为2.38米/秒，风向194度，最大风速4.36米/秒。云层56%。\n\n注意：这里只返回天气描述部分，没有包括其他信息，如“北京”这个城市名称等。如果您需要进一步处理请联系我。'

In [107]:
weather_final.invoke("请问上海天气如何？")

'北京当前天气为晴天，气温30.91℃，湿度56%，风速5.02米/秒。'

In [113]:
# 可以获取上海的天气，但是上述回答的是北京的天气，下面对其进行检测
get_weather('Shanghai')

'{"coord": {"lon": 121.4581, "lat": 31.2222}, "weather": [{"id": 800, "main": "Clear", "description": "\\u6674", "icon": "01d"}], "base": "stations", "main": {"temp": 31.91, "feels_like": 35.72, "temp_min": 31.91, "temp_max": 31.91, "pressure": 999, "humidity": 56, "sea_level": 999, "grnd_level": 998}, "visibility": 10000, "wind": {"speed": 5.02, "deg": 9, "gust": 6.27}, "clouds": {"all": 10}, "dt": 1752484535, "sys": {"country": "CN", "sunrise": 1752440378, "sunset": 1752490797}, "timezone": 28800, "id": 1796236, "name": "Shanghai", "cod": 200}'

In [109]:
weather_model_2.invoke("请问上海的天气如何？")

{'loc': 'Shanghai'}

In [110]:
weather_model_3.invoke("请问上海的天气如何？")

'{"coord": {"lon": 121.4581, "lat": 31.2222}, "weather": [{"id": 800, "main": "Clear", "description": "\\u6674", "icon": "01d"}], "base": "stations", "main": {"temp": 31.91, "feels_like": 35.72, "temp_min": 31.91, "temp_max": 31.91, "pressure": 999, "humidity": 56, "sea_level": 999, "grnd_level": 998}, "visibility": 10000, "wind": {"speed": 5.02, "deg": 9, "gust": 6.27}, "clouds": {"all": 10}, "dt": 1752484535, "sys": {"country": "CN", "sunrise": 1752440378, "sunset": 1752490797}, "timezone": 28800, "id": 1796236, "name": "Shanghai", "cod": 200}'

In [114]:
# 二次对接大模型输出，输出有时候符合有时候不符合，可能与大模型的大小能力有关，小模型的效果一般。
weather_final.invoke("请问上海的天气如何？")

'上海当前天气为晴天，气温31.91℃，湿度56%，风速5.02米/秒'

In [115]:
weather_model_2.invoke("请问利物浦的天气如何？")

{'loc': 'Liverpool'}

In [116]:
weather_final.invoke("请问利物浦的天气如何？")

'"利物浦当前天气为多云，气温21.33℃，湿度63%，风速3.94米/秒，气压1008"'